In [ ]:
import concurrent.futures
from tqdm.auto import tqdm, trange
import time
from openai import AzureOpenAI
import random
import os
import pandas as pd

client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version = "2024-05-01-preview",
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
)

In [ ]:
def gen_responses(prompt_text):

    response = client.chat.completions.create(
      model="gpt4o-2024-05-13",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt_text
            }
          ]
        }
      ],
      temperature=1,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0.04,
      presence_penalty=0.1
    )

    return response


In [ ]:
from FlagEmbedding import BGEM3FlagModel
import torch
import numpy as np
import pandas as pd

def deduplicate_rows(yagi_df):
    model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

    lang_df_list = []
    for language, lang_df in yagi_df.groupby("language"):
        print(language)

        sentences = lang_df.astype(str).prompt_list.tolist()

        embeddings = model.encode(sentences,
                                    batch_size=12,
                                    max_length=8192, 
                                    )['dense_vecs']

        # Move embeddings to GPU and calculate similarity
        prompt_embeddings_cuda = torch.Tensor(embeddings).to("cuda")
        prompt_sim = prompt_embeddings_cuda @ prompt_embeddings_cuda.T
        prompt_sim = prompt_sim.cpu().numpy()

        # Make similarity 0 for top right traingle of similarity matrix so that we only remove one of high similarity pairs
        np.fill_diagonal(prompt_sim, 0)
        prompt_sim *= np.tri(*prompt_sim.shape)

        max_similarity = 0.8
        bad_item_idx_row, bad_item_idx_col = np.where(prompt_sim > 0.8)
        unique_rows = set(np.unique(bad_item_idx_row))

        mask = [True if i not in unique_rows else False for i in range(embeddings.shape[0])]

        deduplicated_lang_df = lang_df[mask]

        lang_df_list.append(deduplicated_lang_df)

    all_df = pd.concat(lang_df_list)

    all_df = all_df[~all_df.prompt_list.duplicated()]
    all_df = all_df.dropna()
    return all_df

In [ ]:
from ftlangdetect import detect

def parse_prompts_from_list(yagi_df):
    yagi_df.loc[:, "prompt_list"] = yagi_df.gen_prompts.str.split("\n").apply(
        lambda x: [y[len(str(i+1)):].strip(" .-。") for i, y in enumerate(x) if y[:len(str(i+1))] == str(i+1)]
    )

    yagi_df.loc[:, "prompt_list"] = yagi_df["prompt_list"].apply(lambda x: [y for y in x if len(y.strip()) > 0])

    yagi_df = yagi_df[['language', 'finish_reason', 'prompt_list']].explode("prompt_list")

    yagi_df.loc[:, "prompt_list"] = yagi_df.prompt_list.str.strip()

    return yagi_df

def is_english_hi_confidence(text):
    lang_res = detect(text=text, low_memory=False)
    return lang_res["lang"] == "en" and lang_res["score"] > 0.8

def is_japanese_hi_confidence(text):
    lang_res = detect(text=text, low_memory=False)
    return lang_res["lang"] == "ja" and lang_res["score"] > 0.8

def filter_out_wrong_languages(yagi_df):
    mask = yagi_df.progress_apply(lambda x: isinstance(x["prompt_list"], str) and is_english_hi_confidence(x["prompt_list"]) and x["language"] not in ["Old English", "Simple English", "English"], axis=1)
    yagi_df = yagi_df[~mask]

    mask = yagi_df.progress_apply(lambda x: isinstance(x["prompt_list"], str) and is_japanese_hi_confidence(x["prompt_list"]) and x["language"] not in ["Japanese"], axis=1)
    yagi_df = yagi_df[~mask]

    return yagi_df

def pre_parse_df(yagi_df, do_dedupe=False):
    yagi_df = parse_prompts_from_list(yagi_df)
    yagi_df = filter_out_wrong_languages(yagi_df)
    
    if do_dedupe:
        yagi_df = deduplicate_rows(yagi_df)
        
    return yagi_df

In [ ]:
from datasets import load_dataset

def run_gen_responses(prompt_text):
    try:
        time.sleep(4 * random.random())
        r = gen_responses(prompt_text)
        return {
            "gen_prompts": r.choices[0].message.content,
            "finish_reason": r.choices[0].finish_reason,
        }
    except Exception as e:
        print(f"Failed - {prompt_text}")
        print(e)
        print()
        return None

def save_dataset(yagi_df, dataset_name):

    chunk_size = 500

    for i in trange(0, yagi_df.shape[0], chunk_size):

        batch_df = yagi_df.iloc[i:i+chunk_size]
        prompt_texts = batch_df["prompt_list"].tolist()

        with concurrent.futures.ThreadPoolExecutor(max_workers=14) as executor:
            response_results = list(tqdm(executor.map(run_gen_responses, prompt_texts), total=len(prompt_texts)))

        batch_df.loc[:, "responses"] = response_results
        batch_df.to_parquet(f"~/{dataset_name}/{str(i).zfill(6)}.parquet")


    yagi_dataset = load_dataset("parquet", data_files={"train": f"~/{dataset_name}/*.parquet"}, split="train")
    yagi_dataset.push_to_hub(f"lightblue/{dataset_name}", private=True)

In [ ]:
from datasets import load_dataset
yagi_297_df = load_dataset("lightblue/yagi_297_preprocessed", split="train").to_pandas().dropna()
yagi_297_df = pre_parse_df(yagi_297_df)

save_dataset(yagi_297_df, "yagi_297")

In [ ]:
from datasets import load_dataset
yagi_58_df = load_dataset("lightblue/yagi_58_preprocessed", split="train").to_pandas().dropna()
yagi_58_df = pre_parse_df(yagi_58_df, do_dedupe=True)

save_dataset(yagi_297_df, "yagi_58")